In [75]:
import os
import pandas as pd
import glob
import nbformat

In [76]:
directory = 'C:/Users/FANNY MARIA D SOUZA/Capstone_Movement/Movement_CompletedSteps_ComplexStew/'
file_pattern = directory + '*_Movement.ipynb'

#### Extracting Position Features

In [77]:
def extract_features_summary(file_path):
    with open(file_path, 'r') as file:
        nb = nbformat.read(file, as_version=4)
    
    local_context = {}
    for cell in nb.cells:
        if cell.cell_type == 'code':
            try:
                # Check if the cell contains plotting code and skip it
                if 'plt.show()' in cell.source or 'plt.plot' in cell.source or 'plt.bar' in cell.source:
                    continue
                exec(cell.source, {}, local_context)
                if 'total_distances_df' in local_context:
                    return local_context['total_distances_df']
            except Exception as e:
                print(f"Error executing cell in {file_path}: {e}")
    return None

total_distance = []

# Extract data from each notebook
for file_path in glob.glob(file_pattern):
    participant_id = os.path.basename(file_path)[:4] 
    df = extract_features_summary(file_path)
    if df is not None:
        df['Participant'] = participant_id 
        total_distance.append(df)
    else:
        print(f"No features_summary found in {file_path}")

combined_features_summary = pd.concat(total_distance, ignore_index=True)
combined_features_summary

,Tracker,Total Distance,Participant
0,Chest,3204.066141,C002
1,LeftShoulder,3149.831729,C002
2,RightShoulder,8647.907569,C002
3,LeftElbow,2980.379981,C002
4,RightElbow,14006.698183,C002
...,...,...,...
115,LeftElbow,7673.767177,C031
116,RightElbow,15793.368219,C031
117,MotionControllerLeft,7747.037142,C031
118,MotionControllerRight,21996.709334,C031


In [78]:
pivoted_df = combined_features_summary.pivot_table(
    index='Participant', 
    columns='Tracker', 
    values='Total Distance'
).reset_index()

pivoted_df


Tracker,Participant,Chest,HeadCamera,LeftElbow,LeftShoulder,MotionControllerLeft,MotionControllerRight,RightElbow,RightShoulder
0,C002,3204.066141,4673.310178,2980.379981,3149.831729,3925.430860,17259.208604,14006.698183,8647.907569
1,C004,2836.300561,5411.832758,8191.224968,7108.797399,11956.925071,9205.327224,5604.074989,5133.590254
2,C005,10539.595320,13065.897281,10215.473433,11062.715877,12582.395922,21996.871609,13598.045673,13520.227658
3,C010,1793.123521,1893.995601,2538.035150,2245.934665,2928.781839,4733.746639,3350.003130,2640.860411
4,C011,4050.538997,4786.556109,6095.937754,5128.255280,7368.995911,13299.942287,8518.602496,6479.545112
5,C014,4973.936621,6028.289712,6796.033200,6214.696710,7591.382218,15144.486600,11593.240494,9524.565936
6,C015,7321.705980,9195.631796,10355.253717,9531.107151,10771.533045,22361.696808,17257.506265,12847.732522
7,C018,3278.754839,4263.423725,4815.631482,4005.065979,6542.016552,10215.028963,7051.163868,5527.008737
8,C019,3145.024030,4478.292426,3391.126102,3281.582209,4376.936796,13280.100424,10453.141934,7320.194072
9,C022,3408.740260,4343.240819,4916.747164,4590.815735,5278.099917,11001.978297,7552.145704,6045.832878


In [79]:
from sklearn.impute import KNNImputer
import numpy as np

imputer = KNNImputer(n_neighbors=1)

numerical_cols = pivoted_df.select_dtypes(include=['float64', 'int64']).columns
pivoted_df[numerical_cols] = pivoted_df[numerical_cols].replace(0, np.nan)

pivoted_df[numerical_cols] = imputer.fit_transform(pivoted_df[numerical_cols])
pivoted_df

Tracker,Participant,Chest,HeadCamera,LeftElbow,LeftShoulder,MotionControllerLeft,MotionControllerRight,RightElbow,RightShoulder
0,C002,3204.066141,4673.310178,2980.379981,3149.831729,3925.430860,17259.208604,14006.698183,8647.907569
1,C004,2836.300561,5411.832758,8191.224968,7108.797399,11956.925071,9205.327224,5604.074989,5133.590254
2,C005,10539.595320,13065.897281,10215.473433,11062.715877,12582.395922,21996.871609,13598.045673,13520.227658
3,C010,1793.123521,1893.995601,2538.035150,2245.934665,2928.781839,4733.746639,3350.003130,2640.860411
4,C011,4050.538997,4786.556109,6095.937754,5128.255280,7368.995911,13299.942287,8518.602496,6479.545112
5,C014,4973.936621,6028.289712,6796.033200,6214.696710,7591.382218,15144.486600,11593.240494,9524.565936
6,C015,7321.705980,9195.631796,10355.253717,9531.107151,10771.533045,22361.696808,17257.506265,12847.732522
7,C018,3278.754839,4263.423725,4815.631482,4005.065979,6542.016552,10215.028963,7051.163868,5527.008737
8,C019,3145.024030,4478.292426,3391.126102,3281.582209,4376.936796,13280.100424,10453.141934,7320.194072
9,C022,3408.740260,4343.240819,4916.747164,4590.815735,5278.099917,11001.978297,7552.145704,6045.832878


In [80]:
new_column_names = {
    'Chest': 'Chest Total Distance',
    'HeadCamera': 'HeadCamera Total Distance',
    'LeftElbow': 'LeftElbow Total Distance',
    'LeftShoulder': 'LeftShoulder Total Distance',
    'MotionControllerLeft': 'MotionControllerLeft Total Distance',
    'MotionControllerRight': 'MotionControllerRight Total Distance',
    'RightElbow': 'RightElbow Total Distance',
    'RightShoulder': 'RightShoulder Total Distance'
}

# Rename columns
pivoted_df = pivoted_df.rename(columns=new_column_names)
pivoted_df

Tracker,Participant,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance
0,C002,3204.066141,4673.310178,2980.379981,3149.831729,3925.430860,17259.208604,14006.698183,8647.907569
1,C004,2836.300561,5411.832758,8191.224968,7108.797399,11956.925071,9205.327224,5604.074989,5133.590254
2,C005,10539.595320,13065.897281,10215.473433,11062.715877,12582.395922,21996.871609,13598.045673,13520.227658
3,C010,1793.123521,1893.995601,2538.035150,2245.934665,2928.781839,4733.746639,3350.003130,2640.860411
4,C011,4050.538997,4786.556109,6095.937754,5128.255280,7368.995911,13299.942287,8518.602496,6479.545112
5,C014,4973.936621,6028.289712,6796.033200,6214.696710,7591.382218,15144.486600,11593.240494,9524.565936
6,C015,7321.705980,9195.631796,10355.253717,9531.107151,10771.533045,22361.696808,17257.506265,12847.732522
7,C018,3278.754839,4263.423725,4815.631482,4005.065979,6542.016552,10215.028963,7051.163868,5527.008737
8,C019,3145.024030,4478.292426,3391.126102,3281.582209,4376.936796,13280.100424,10453.141934,7320.194072
9,C022,3408.740260,4343.240819,4916.747164,4590.815735,5278.099917,11001.978297,7552.145704,6045.832878


In [81]:
pivoted_df.to_csv('PositionDistance_ComplexStew.csv', index=False)

In [82]:
bscore=pd.read_csv('M_BDScore.csv')

In [83]:
final_df = pd.merge(bscore, pivoted_df, on='Participant', how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance
0,C002,28,3204.066141,4673.310178,2980.379981,3149.831729,3925.430860,17259.208604,14006.698183,8647.907569
1,C004,30,2836.300561,5411.832758,8191.224968,7108.797399,11956.925071,9205.327224,5604.074989,5133.590254
2,C005,46,10539.595320,13065.897281,10215.473433,11062.715877,12582.395922,21996.871609,13598.045673,13520.227658
3,C010,54,1793.123521,1893.995601,2538.035150,2245.934665,2928.781839,4733.746639,3350.003130,2640.860411
4,C011,34,4050.538997,4786.556109,6095.937754,5128.255280,7368.995911,13299.942287,8518.602496,6479.545112
5,C014,32,4973.936621,6028.289712,6796.033200,6214.696710,7591.382218,15144.486600,11593.240494,9524.565936
6,C015,33,7321.705980,9195.631796,10355.253717,9531.107151,10771.533045,22361.696808,17257.506265,12847.732522
7,C018,50,3278.754839,4263.423725,4815.631482,4005.065979,6542.016552,10215.028963,7051.163868,5527.008737
8,C019,40,3145.024030,4478.292426,3391.126102,3281.582209,4376.936796,13280.100424,10453.141934,7320.194072
9,C022,32,3408.740260,4343.240819,4916.747164,4590.815735,5278.099917,11001.978297,7552.145704,6045.832878


In [84]:
addbeef=pd.read_csv('ComplexStew_AddBeef.csv')

In [85]:
if 'Block Design Raw Score' in addbeef.columns:
    addbeef['Block Design Raw Score'] = addbeef['Block Design Raw Score'].astype(int)
addbeef = addbeef.drop(columns=['Step'])

final_df = pd.merge(final_df, addbeef, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,Chest Distance - Add Beef,HeadCamera Distance - Add Beef,LeftElbow Distance - Add Beef,LeftShoulder Distance - Add Beef,MotionControllerLeft Distance - Add Beef,MotionControllerRight Distance - Add Beef,RightElbow Distance - Add Beef,RightShoulder Distance - Add Beef
0,C002,28,3204.066141,4673.310178,2980.379981,3149.831729,3925.430860,17259.208604,14006.698183,8647.907569,540.634569,871.385365,616.627310,551.960792,870.054407,3283.678374,2454.728468,1567.848626
1,C004,30,2836.300561,5411.832758,8191.224968,7108.797399,11956.925071,9205.327224,5604.074989,5133.590254,678.021052,1056.024777,1333.029026,1224.846997,1953.444900,1969.429199,1167.451003,1245.300403
2,C005,46,10539.595320,13065.897281,10215.473433,11062.715877,12582.395922,21996.871609,13598.045673,13520.227658,527.974440,676.804793,1141.001602,798.177675,1330.338188,1761.635707,1366.876046,1110.322250
3,C011,34,4050.538997,4786.556109,6095.937754,5128.255280,7368.995911,13299.942287,8518.602496,6479.545112,757.515406,902.667677,998.559476,844.998041,907.629452,3065.018582,1924.378370,1372.717540
4,C015,33,7321.705980,9195.631796,10355.253717,9531.107151,10771.533045,22361.696808,17257.506265,12847.732522,1309.921467,1754.988121,1829.264137,1805.224913,1682.359192,4344.701474,3481.695626,2629.277680
5,C018,50,3278.754839,4263.423725,4815.631482,4005.065979,6542.016552,10215.028963,7051.163868,5527.008737,507.964028,846.421903,512.605757,566.642162,606.725382,2240.851043,1523.371874,1162.647533
6,C019,40,3145.024030,4478.292426,3391.126102,3281.582209,4376.936796,13280.100424,10453.141934,7320.194072,407.425461,587.880202,397.311728,398.836634,1416.035533,1755.487755,1277.206584,860.739231
7,C023,52,5663.025763,6679.813291,7694.291884,6896.760661,8494.780272,17624.559759,13460.745772,9701.709138,1575.045027,1575.630306,1833.642797,1666.449108,2334.444410,4672.175111,3326.770177,2456.115983
8,C026,33,2914.637327,4807.716956,2926.516585,3449.654787,2798.885941,15821.240910,13939.200526,8656.802318,306.848860,564.804134,302.741243,384.603963,226.258566,1788.584405,1507.303911,1016.467496
9,C027,41,4099.889010,4006.935510,5427.315485,4531.051608,4987.657671,12565.330603,9632.380454,7365.477357,506.024903,557.296842,680.568540,661.776561,608.798586,2185.573133,1641.029995,1203.192888


In [86]:
addcarrot=pd.read_csv('ComplexStew_AddCarrot.csv')

In [87]:
if 'Block Design Raw Score' in addcarrot.columns:
    addcarrot['Block Design Raw Score'] = addcarrot['Block Design Raw Score'].astype(int)
addcarrot = addcarrot.drop(columns=['Step'])

final_df = pd.merge(final_df, addcarrot, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Beef,RightShoulder Distance - Add Beef,Chest Distance - Add Carrot,HeadCamera Distance - Add Carrot,LeftElbow Distance - Add Carrot,LeftShoulder Distance - Add Carrot,MotionControllerLeft Distance - Add Carrot,MotionControllerRight Distance - Add Carrot,RightElbow Distance - Add Carrot,RightShoulder Distance - Add Carrot
0,C002,28,3204.066141,4673.310178,2980.379981,3149.831729,3925.430860,17259.208604,14006.698183,8647.907569,...,2454.728468,1567.848626,489.520823,696.793809,443.371566,438.114497,587.927154,3007.709395,2211.665362,1453.483221
1,C004,30,2836.300561,5411.832758,8191.224968,7108.797399,11956.925071,9205.327224,5604.074989,5133.590254,...,1167.451003,1245.300403,509.738419,686.761077,1475.509102,1198.633244,2128.896733,967.129351,649.044687,580.652897
2,C005,46,10539.595320,13065.897281,10215.473433,11062.715877,12582.395922,21996.871609,13598.045673,13520.227658,...,1366.876046,1110.322250,324.531220,439.232042,876.385788,613.136340,1101.918260,1140.626196,144.213006,368.989996
3,C015,33,7321.705980,9195.631796,10355.253717,9531.107151,10771.533045,22361.696808,17257.506265,12847.732522,...,3481.695626,2629.277680,1186.960102,1423.478153,1888.012667,1548.164236,2091.734313,3677.512173,2910.438663,2137.740781
4,C018,50,3278.754839,4263.423725,4815.631482,4005.065979,6542.016552,10215.028963,7051.163868,5527.008737,...,1523.371874,1162.647533,274.056490,456.994332,844.245766,593.923184,1323.123445,1041.964698,670.232599,553.208020
5,C019,40,3145.024030,4478.292426,3391.126102,3281.582209,4376.936796,13280.100424,10453.141934,7320.194072,...,1277.206584,860.739231,338.635267,462.231591,320.211109,318.678709,199.437394,1763.539803,1229.986636,920.463926
6,C023,52,5663.025763,6679.813291,7694.291884,6896.760661,8494.780272,17624.559759,13460.745772,9701.709138,...,3326.770177,2456.115983,1148.815241,1283.685937,1873.325753,1582.975662,2006.458525,3581.829647,2818.552211,2113.810533
7,C026,33,2914.637327,4807.716956,2926.516585,3449.654787,2798.885941,15821.240910,13939.200526,8656.802318,...,1507.303911,1016.467496,302.945933,465.072978,223.698081,292.922785,162.038118,2131.436409,1905.742975,1209.620948
8,C027,41,4099.889010,4006.935510,5427.315485,4531.051608,4987.657671,12565.330603,9632.380454,7365.477357,...,1641.029995,1203.192888,515.044701,536.607618,734.329636,627.317603,640.072857,2337.766837,1700.032890,1241.011001
9,C031,61,7321.705980,7888.943737,7673.767177,7619.326954,7747.037142,21996.709334,15793.368219,11124.952609,...,3893.843454,2659.180097,515.044701,569.619857,625.937927,608.833140,580.985634,2057.390420,1505.495664,1009.712497


In [88]:
addtomato=pd.read_csv('ComplexStew_AddTomato.csv')

In [89]:
if 'Block Design Raw Score' in addtomato.columns:
    addtomato['Block Design Raw Score'] = addtomato['Block Design Raw Score'].astype(int)
addtomato = addtomato.drop(columns=['Step'])

final_df = pd.merge(final_df, addtomato, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Carrot,RightShoulder Distance - Add Carrot,Chest Distance - Add Tomato,HeadCamera Distance - Add Tomato,LeftElbow Distance - Add Tomato,LeftShoulder Distance - Add Tomato,MotionControllerLeft Distance - Add Tomato,MotionControllerRight Distance - Add Tomato,RightElbow Distance - Add Tomato,RightShoulder Distance - Add Tomato
0,C002,28,3204.066141,4673.310178,2980.379981,3149.831729,3925.430860,17259.208604,14006.698183,8647.907569,...,2211.665362,1453.483221,484.808395,662.174721,447.707068,451.546312,611.794963,2483.162082,1970.630517,1292.022854
1,C004,30,2836.300561,5411.832758,8191.224968,7108.797399,11956.925071,9205.327224,5604.074989,5133.590254,...,649.044687,580.652897,369.266755,504.991223,629.273380,580.831621,821.496227,984.074168,467.556067,553.625484
2,C005,46,10539.595320,13065.897281,10215.473433,11062.715877,12582.395922,21996.871609,13598.045673,13520.227658,...,144.213006,368.989996,384.638898,442.044766,709.030236,512.652148,803.767682,1303.964618,328.266407,185.133009
3,C015,33,7321.705980,9195.631796,10355.253717,9531.107151,10771.533045,22361.696808,17257.506265,12847.732522,...,2910.438663,2137.740781,518.639895,629.785955,857.309903,728.891950,914.524496,1572.341149,1079.360173,811.571043
4,C018,50,3278.754839,4263.423725,4815.631482,4005.065979,6542.016552,10215.028963,7051.163868,5527.008737,...,670.232599,553.208020,736.712759,626.404064,441.994578,450.783926,536.759991,1859.820871,1210.445885,892.544275
5,C019,40,3145.024030,4478.292426,3391.126102,3281.582209,4376.936796,13280.100424,10453.141934,7320.194072,...,1229.986636,920.463926,260.648946,354.727375,182.358952,217.255372,137.192101,1139.756740,919.206474,586.956247
6,C023,52,5663.025763,6679.813291,7694.291884,6896.760661,8494.780272,17624.559759,13460.745772,9701.709138,...,2818.552211,2113.810533,537.542271,665.137118,780.193830,728.078605,666.732741,1764.158239,1386.871638,1048.351633
7,C026,33,2914.637327,4807.716956,2926.516585,3449.654787,2798.885941,15821.240910,13939.200526,8656.802318,...,1905.742975,1209.620948,237.631869,436.841251,220.550201,287.905830,141.918255,1442.885913,1778.344727,774.524017
8,C027,41,4099.889010,4006.935510,5427.315485,4531.051608,4987.657671,12565.330603,9632.380454,7365.477357,...,1700.032890,1241.011001,519.075144,585.267884,953.455647,592.562666,587.783344,1670.827940,1365.531432,1440.582213
9,C031,61,7321.705980,7888.943737,7673.767177,7619.326954,7747.037142,21996.709334,15793.368219,11124.952609,...,1505.495664,1009.712497,736.712759,635.167438,548.435997,579.916351,492.380788,1830.373445,1284.401408,884.655523


In [90]:
pourbroth=pd.read_csv('ComplexStew_PourBroth.csv')

In [91]:
if 'Block Design Raw Score' in pourbroth.columns:
    pourbroth['Block Design Raw Score'] = pourbroth['Block Design Raw Score'].astype(int)
pourbroth = pourbroth.drop(columns=['Step'])

final_df = pd.merge(final_df, pourbroth, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Add Tomato,RightShoulder Distance - Add Tomato,Chest Distance - Pour Broth,HeadCamera Distance - Pour Broth,LeftElbow Distance - Pour Broth,LeftShoulder Distance - Pour Broth,MotionControllerLeft Distance - Pour Broth,MotionControllerRight Distance - Pour Broth,RightElbow Distance - Pour Broth,RightShoulder Distance - Pour Broth
0,C002,28,3204.066141,4673.310178,2980.379981,3149.831729,3925.430860,17259.208604,14006.698183,8647.907569,...,1970.630517,1292.022854,373.151763,506.636411,388.627938,495.272389,409.224437,725.838103,1714.531014,543.516110
1,C004,30,2836.300561,5411.832758,8191.224968,7108.797399,11956.925071,9205.327224,5604.074989,5133.590254,...,467.556067,553.625484,10.117832,16.713581,12.120759,10.381539,15.261553,23.040013,34.887967,13.121554
2,C005,46,10539.595320,13065.897281,10215.473433,11062.715877,12582.395922,21996.871609,13598.045673,13520.227658,...,328.266407,185.133009,6096.489278,7137.723438,1915.627907,3666.335626,3107.682870,5926.886300,4252.288762,4541.550987
3,C015,33,7321.705980,9195.631796,10355.253717,9531.107151,10771.533045,22361.696808,17257.506265,12847.732522,...,1079.360173,811.571043,597.687203,557.326678,687.310669,699.179024,716.300350,1156.818252,919.768936,761.331724
4,C018,50,3278.754839,4263.423725,4815.631482,4005.065979,6542.016552,10215.028963,7051.163868,5527.008737,...,1210.445885,892.544275,277.328901,340.446999,368.499565,353.665747,400.892510,534.811978,404.994406,398.034021
5,C019,40,3145.024030,4478.292426,3391.126102,3281.582209,4376.936796,13280.100424,10453.141934,7320.194072,...,919.206474,586.956247,752.172470,972.534955,1232.246213,983.651842,1599.751829,1307.462087,1625.807197,1234.110161
6,C023,52,5663.025763,6679.813291,7694.291884,6896.760661,8494.780272,17624.559759,13460.745772,9701.709138,...,1386.871638,1048.351633,596.604117,1007.959245,828.836547,806.964682,999.265969,1255.790276,1064.937918,707.681496
7,C026,33,2914.637327,4807.716956,2926.516585,3449.654787,2798.885941,15821.240910,13939.200526,8656.802318,...,1778.344727,774.524017,513.534038,752.682182,697.409454,565.527539,995.217865,1584.682746,1236.407720,788.489183
8,C027,41,4099.889010,4006.935510,5427.315485,4531.051608,4987.657671,12565.330603,9632.380454,7365.477357,...,1365.531432,1440.582213,545.894624,184.727824,424.240630,330.016112,415.668478,643.389642,441.419751,385.521093
9,C031,61,7321.705980,7888.943737,7673.767177,7619.326954,7747.037142,21996.709334,15793.368219,11124.952609,...,1284.401408,884.655523,752.172470,2148.540098,2298.817658,2170.840187,2575.201670,3188.117916,2396.057172,2058.860757


In [92]:
stirstew=pd.read_csv('ComplexStew_StirStew.csv')

In [93]:
if 'Block Design Raw Score' in stirstew.columns:
    stirstew['Block Design Raw Score'] = stirstew['Block Design Raw Score'].astype(int)
stirstew = stirstew.drop(columns=['Step'])

final_df = pd.merge(final_df, stirstew, on=['Participant', 'Block Design Raw Score'], how='inner')
final_df

,Participant,Block Design Raw Score,Chest Total Distance,HeadCamera Total Distance,LeftElbow Total Distance,LeftShoulder Total Distance,MotionControllerLeft Total Distance,MotionControllerRight Total Distance,RightElbow Total Distance,RightShoulder Total Distance,...,RightElbow Distance - Pour Broth,RightShoulder Distance - Pour Broth,Chest Distance - Stir Stew,HeadCamera Distance - Stir Stew,LeftElbow Distance - Stir Stew,LeftShoulder Distance - Stir Stew,MotionControllerLeft Distance - Stir Stew,MotionControllerRight Distance - Stir Stew,RightElbow Distance - Stir Stew,RightShoulder Distance - Stir Stew
0,C002,28,3204.066141,4673.310178,2980.379981,3149.831729,3925.430860,17259.208604,14006.698183,8647.907569,...,1714.531014,543.516110,114.036756,182.004488,75.462707,106.145378,75.784500,710.157202,437.372259,334.352882
1,C004,30,2836.300561,5411.832758,8191.224968,7108.797399,11956.925071,9205.327224,5604.074989,5133.590254,...,34.887967,13.121554,56.849798,143.411908,54.108509,76.926598,21.515681,460.668195,303.000311,208.132135
2,C005,46,10539.595320,13065.897281,10215.473433,11062.715877,12582.395922,21996.871609,13598.045673,13520.227658,...,4252.288762,4541.550987,156.942825,395.658365,499.920146,563.429103,270.665309,1118.133664,653.662983,316.679881
3,C015,33,7321.705980,9195.631796,10355.253717,9531.107151,10771.533045,22361.696808,17257.506265,12847.732522,...,919.768936,761.331724,347.806183,507.724735,432.090842,414.135636,458.578783,1190.581168,1116.672396,764.177645
4,C018,50,3278.754839,4263.423725,4815.631482,4005.065979,6542.016552,10215.028963,7051.163868,5527.008737,...,404.994406,398.034021,56.849798,122.808691,54.108509,53.553696,93.753007,455.950906,282.756327,180.778963
5,C019,40,3145.024030,4478.292426,3391.126102,3281.582209,4376.936796,13280.100424,10453.141934,7320.194072,...,1625.807197,1234.110161,71.582511,140.869081,54.108509,83.396543,178.080206,478.542995,318.821234,227.718322
6,C023,52,5663.025763,6679.813291,7694.291884,6896.760661,8494.780272,17624.559759,13460.745772,9701.709138,...,1064.937918,707.681496,111.778657,149.604708,150.357421,132.127479,170.026629,585.104100,406.843985,226.287122
7,C026,33,2914.637327,4807.716956,2926.516585,3449.654787,2798.885941,15821.240910,13939.200526,8656.802318,...,1236.407720,788.489183,175.164592,320.458210,172.334591,245.443317,143.912354,1265.417759,925.314665,605.216266
8,C027,41,4099.889010,4006.935510,5427.315485,4531.051608,4987.657671,12565.330603,9632.380454,7365.477357,...,441.419751,385.521093,294.041869,321.837286,351.531484,331.585487,356.460480,967.750208,818.337178,500.231373
9,C031,61,7321.705980,7888.943737,7673.767177,7619.326954,7747.037142,21996.709334,15793.368219,11124.952609,...,2396.057172,2058.860757,111.778657,223.546864,160.462614,175.309532,183.507411,668.240500,442.039163,304.221958


In [94]:
final_df.to_csv('Features_ComplexStew_Movement.csv',index=False)